In [1]:
import os
!pwd

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/intent_classification/research


In [2]:
# Goto Root directory of the project
os.chdir("../")
!pwd

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/intent_classification


In [14]:
# Model trauiner Config Schema/Data Class
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    output_dir: Path
    logging_dir: Path
    eval_strategy: str
    learning_rate: float
    per_device_train_batch_size: int
    num_train_epochs: int
    weight_decay: float
    logging_steps: int

In [15]:
from src.textClassifier.constants import *
from src.textClassifier.utils.common import read_yaml, create_directories
from box import ConfigBox

## Config Manager

In [16]:
# Create Configuration Manager: This has basic information required before staring any module/components

class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config:ConfigBox = read_yaml(config_path)
        self.params:ConfigBox = read_yaml(params_path)

        # Create Artifacts Root dir
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            output_dir=self.params.TrainingArguments.output_dir,
            logging_dir=self.params.TrainingArguments.logging_dir,
            eval_strategy=self.params.TrainingArguments.eval_strategy,
            learning_rate=self.params.TrainingArguments.learning_rate,
            per_device_train_batch_size=self.params.TrainingArguments.per_device_train_batch_size,
            num_train_epochs=self.params.TrainingArguments.num_train_epochs,
            weight_decay=self.params.TrainingArguments.weight_decay,
            logging_steps=self.params.TrainingArguments.logging_steps
        )

        return model_trainer_config


In [17]:
## Model Trainer libraries import
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from transformers import TrainingArguments, Trainer
import torch
from datasets import load_from_disk

## Model Trainer component

In [18]:
class ModelTrainer:
    def __init__(self, model_trainer_config: ModelTrainerConfig):
        self.config = model_trainer_config

    # Training logic
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = DistilBertTokenizer.from_pretrained(self.config.model_ckpt)

        model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=8)

        # loading the data
        encoded_dataset = load_from_disk(self.config.data_path)

        training_args = TrainingArguments(
            output_dir="./results",           # Directory for saving results
            eval_strategy="epoch",     # Evaluate at the end of each epoch
            learning_rate=5e-5,              # Initial learning rate
            per_device_train_batch_size=16,  # Batch size per GPU
            num_train_epochs=3,              # Number of epochs
            weight_decay=0.01,               # Regularization
            logging_dir="./logs",            # Directory for logs
            logging_steps=10                 # Log every 10 steps
        )

        trainer = Trainer(
            model=model,                          # The DistilBERT model
            args=training_args,                   # Training arguments
            train_dataset=encoded_dataset['train'],  # Training data
            eval_dataset=encoded_dataset['validation']  # Validation data
        )

        # Start training
        trainer.train()

        ## Save model
        model.save_pretrained("./finetuned_model")
        ## Save tokenizer
        tokenizer.save_pretrained("./finetuned_model")

In [19]:
# Pipeline initiate

config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()

model_trainer = ModelTrainer(model_trainer_config)

model_trainer.train()

[2025-05-10 23:20:59,925 : INFO: common: yaml file: config/config.yaml is loaded sucessfully]
[2025-05-10 23:20:59,927 : INFO: common: yaml file: params.yaml is loaded sucessfully]
[2025-05-10 23:20:59,928 : INFO: common: Created directory at: artifacts]
[2025-05-10 23:20:59,929 : INFO: common: Created directory at: artifacts/model_trainer]


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: tvggamermax (tvggamermax-na) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/intent_classification/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 